# Importaciones

In [ ]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import requests

# Leyendo los datos

Utilizaremos los datos de las solicitudes al INAI, por ser un dataset con más tipos de datos y otro posible tipo de complicaciones.

In [ ]:
data_raw = requests.get('https://api.datos.gob.mx/v1/inai.solicitudes')
data_raw

In [ ]:
data_raw = requests.get('https://api.datos.gob.mx/v1/inai.solicitudes')
data = data_raw.text
json_data = json.loads(data)
pandas_data = pd.json_normalize(json_data['results'])
pandas_data.head(5)

Tal como lo vimos en la clase de EDA, el objetivo de realizar este tipo de análisis es conocer los datos con los que resolveremos un problema y saber cuáles son las transformaciones que haremos en la etapa de Transform de un pipeline de datos.

Primero, conoceremos el dataset completo haciendo una exploración de:
* Número de variables
* Número de renglones u observaciones
* Número de observaciones únicas por variable
* ¿Qué variables tenemos?
* Número de variables numéricas
* Número de variables de fecha
* Número de variables categóricas
* Número de variables de texto
* Número de nulos

In [ ]:
## Número de variables y Número de renglones
shpe = pandas_data.shape
print("Contamos con ",shpe[0]," renglones y", shpe[1], " columnas.")

In [ ]:
# Número de variables únicas por cada columna
num_unique_vars = pd.DataFrame(pandas_data.nunique())
num_unique_vars.columns = ['count_of_unique_variables']
num_unique_vars

In [ ]:
# ¿Qué variables tenemos?
list(pandas_data.columns)

**Ojo!!!** Aqui podemos observar que tenemos una variable cuyo nombre de columna está vacío y que tiene un solo valor único. En la etapa de Transform, este tipo de datos debe ser verificado o eliminado o corregido.

In [ ]:
# Tipos de datos de cada variable.
data_types = pd.DataFrame(pandas_data.dtypes)
data_types.columns = ['Data Type']
data_types

Aqui vemos que todas las variables nos aparecen como strings o texto.   
¿Es esto real?   
¿No hay fechas?  
¿No hay variables numéricas?  


In [ ]:
# Conteo de valores nulos
pandas_data.isna().sum()

### ¿Qué contiene cada variable?

¿Qué tiene la columna sin nombre?

In [ ]:
pandas_data[''].unique()

Está vacía, por lo tanto podríamos deshacernos de ella en la etapa del Transform.

Existe la columna `_id` y la columna Folio. ¿Cuál es el identificador de cada registro?

In [ ]:
pandas_data['_id'].unique()

In [ ]:
pandas_data['FOLIO'].unique()

Este es el tipo de cosas que podría llegar a preguntársele al área de negocio, pero en este caso, no es la primera vez que ocupo este dataset, por lo que yo ya sé que `_id` nos indica el id de cuando fue cargada la información, mientras que `FOLIO`indica el identificador en sistemas de cada solicitud.

Adicionalmente, a pesar de que `FOLIO` son una serie de números, sabemos que no tiene sentido convertirlo en una variable numérica, ya que es solo un identificador del registro.

Lo que sucede con la columna `_id` es que en ocasiones, se pone un identificador de este tipo para saber si ya tienes o no este registro en tu base de datos, en caso de que quieras utilizarla como un input que se va registrando de forma histórica.

#### Fecha Solicitud:

In [ ]:
pandas_data['FECHASOLICITUD'].unique()

Esta variable contiene fechas, por lo que debe ser transformada en una variable de ese tipo.

#### Dependencia:

In [ ]:
pandas_data['DEPENDENCIA'].unique()

#### Estatus

In [ ]:
pandas_data['ESTATUS'].unique()

#### Medio entrada:

In [ ]:
pandas_data['MEDIOENTRADA'].unique()

####  Tipo Solicitud:

In [ ]:
pandas_data['TIPOSOLICITUD'].unique()

#### Descripción solicitud:

In [ ]:
pandas_data['DESCRIPCIONSOLICITUD'].unique()

#### Otros Datos:

In [ ]:
pandas_data['OTROSDATOS'].unique()

#### Archivo adjunto solicitud

In [ ]:
pandas_data['ARCHIVOADJUNTOSOLICITUD'].unique()

#### Medio Entrega

In [ ]:
pandas_data['MEDIOENTREGA'].unique()

#### Fecha Limite

In [ ]:
pandas_data['FECHALIMITE'].unique()

#### Respuesta

In [ ]:
pandas_data['RESPUESTA'].unique()

#### Texto Respuesta

In [ ]:
pandas_data['TEXTORESPUESTA'].unique()

#### Archivo Respuesta

In [ ]:
pandas_data['ARCHIVORESPUESTA'].unique()

#### Fecha Respuesta:

In [ ]:
pandas_data['FECHARESPUESTA'].unique()

#### Pais

In [ ]:
pandas_data['PAIS'].unique()

#### Estado

In [ ]:
pandas_data['ESTADO'].unique()

#### Municipio

In [ ]:
pandas_data['MUNICIPIO'].unique()

#### Codigo Postal

In [ ]:
pandas_data['CODIGOPOSTAL'].unique()

In [ ]:
codigo_postal_cero = pandas_data.loc[pandas_data['CODIGOPOSTAL']=='0']
codigo_postal_cero

In [ ]:
codigo_postal_uno = pandas_data.loc[pandas_data['CODIGOPOSTAL']=='1']
codigo_postal_uno

#### Sector

In [ ]:
pandas_data['SECTOR'].unique()

¿Qué sabemos del dataset?

* Contamos con  100  renglones y 22  columnas.
* Todos los nombres de columnas están en mayúsculas. --> Si negocio lo solicita, podrían dejarse todos en minúsculas.
* Tenemos 3 variables que tienen solo un valor, pero una de ellas es nombre de columna vacío y contenido vacío, por lo tanto habría que quitarla.
* Tenemos 3 variables que se están tomando como de tipo String (o texto), pero deben estar en formato de fecha o timestamp.
* En el conteo de NA´s nos sale que no hay ninguno. Sin embargo, al revisar los datos a detalle podemos ver que estos nulos están representados con vacío (''), por lo que esto deberá ser cambiado en la etapa de Transform.
* Dos variables contienen ligas a páginas de internet. Podríamos suponer, con fines de hacer una transformación más, que se quiere mantener esta información privada, para que no todos tuvieran acceso a ella.
* La columna `_id` no necesariamente debería llegar al usuario final, pues es un indicador más para quien hace la extracción de los datos que de quien hace analítica de datos.
* Las columnas de tipo texto tienen acentos, caracteres especiales, Altas y Bajas.
* Si negocio lo solicitara, podría requerirse quitar las columnas con una sola variable, Este tipo de información debe ser validada con el área de negocio.
* En el caso de la variable `CODIGOPOSTAL` podríamos sacar dos conclusiones: 
    1. el valor que se utiliza para números nulos es el 0
    2. podría parecer que debe ser numérica, pero en realidad debe ser de tipo String, pues no tiene sentido usarla como numérica.
* En cuanto a la variable `FECHALIMITE`, si el plazo fuera uno fijo establecido se podría inputar desde la fecha de solicitud, sumandole los días que se tienen como fecha límite.
* En la variable `TEXTORESPUESTA`, se tienen algunos datos duplicados que parecería que no lo están. Para esto, podría ser necesario hacer una modificación manualmente. 

![image.png](../../imagenes/eda_images/eda_img_01.jpg)


Toda esta información deberá ser utilizada en la etapa de transform para limpiar y estandarizar los datos.